### Import stuff

In [53]:
import pandas as pd
import numpy as np

### Bring in data

In [7]:
PATH = r"C:\Users\pablo.vega-behar\Desktop\Coding-Scripting\Kaggle\Recruit-restaurant-visitor-forecasting"

In [11]:
data = {
    'airRes':    pd.read_csv(PATH + r"\air_reserve.csv"),
    'airStore':  pd.read_csv(PATH + r"\air_store_info.csv"),
    'airVisit':  pd.read_csv(PATH + r"\air_visit_data.csv"),
    'date':      pd.read_csv(PATH + r"\date_info.csv"),
    'hpgRes':    pd.read_csv(PATH + r"\hpg_reserve.csv"),
    'hpgStore':  pd.read_csv(PATH + r"\hpg_store_info.csv"),
    'sampleSub': pd.read_csv(PATH + r"\sample_submission.csv"),
    'storeIDs':  pd.read_csv(PATH + r"\store_id_relation.csv")    
}

### Check out the data

In [29]:
for (k, v) in data.items():
    print(k)
    print(v.head(1))
    print('_'*30)

airVisit
           air_store_id  visit_date  visitors
0  air_ba937bf13d40fb24  2016-01-13        25
______________________________
sampleSub
                                id  visitors
0  air_00a91d42b08b08d9_2017-04-23         0
______________________________
hpgRes
           hpg_store_id       visit_datetime     reserve_datetime  \
0  hpg_c63f6f42e088e50f  2016-01-01 11:00:00  2016-01-01 09:00:00   

   reserve_visitors  
0                 1  
______________________________
airRes
           air_store_id       visit_datetime     reserve_datetime  \
0  air_877f79706adbfb06  2016-01-01 19:00:00  2016-01-01 16:00:00   

   reserve_visitors  
0                 1  
______________________________
date
  calendar_date day_of_week  holiday_flg
0    2016-01-01      Friday            1
______________________________
hpgStore
           hpg_store_id  hpg_genre_name                  hpg_area_name  \
0  hpg_6622b62385aec8bf  Japanese style  Tōkyō-to Setagaya-ku Taishidō   

    latitude   long

# The dataframes need to be merged
Below I add air ID to hpg Res df.

NOTE: Right now I'm doing an inner join, but not all restaurants are in both airRes and hpgRes!

In [85]:
data['hpgRes'] = pd.merge(data['hpgRes'], data['storeIDs'], how = 'inner', on=['hpg_store_id'])

In [86]:
data['hpgRes'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id_x,visit_date,visit_time,reserve_date,reserve_time,reserve_visit_diff,air_store_id_y
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016-01-01,19:00:00,2016-01-01,15:00:00,0,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926,2016-01-02,19:00:00,2016-01-02,14:00:00,0,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926,2016-01-03,18:00:00,2016-01-02,20:00:00,1,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926,2016-01-06,20:00:00,2016-01-04,22:00:00,2,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926,2016-01-11,18:00:00,2016-01-11,14:00:00,0,air_db80363d35f10926


Separate date and time info for Res df's.

Add reservation-to-visit difference (may not be a feature, as it's leakage)

In [61]:

for dfName in ['airRes', 'hpgRes']:
    data[dfName]['visit_datetime'] = pd.to_datetime(data[dfName]['visit_datetime'])
    data[dfName]['reserve_datetime'] = pd.to_datetime(data[dfName]['reserve_datetime'])
    data[dfName]['visit_date'] = data[dfName]['visit_datetime'].dt.date
    data[dfName]['visit_time'] = data[dfName]['visit_datetime'].dt.time
    data[dfName]['reserve_date'] = data[dfName]['reserve_datetime'].dt.date
    data[dfName]['reserve_time'] = data[dfName]['reserve_datetime'].dt.time
    data[dfName]['reserve_visit_diff'] = data[dfName].apply(lambda x: (x['visit_date'] - x['reserve_date']).days, axis = 1)

data['airRes'].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,visit_time,reserve_date,reserve_time,reserve_visit_diff
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,19:00:00,2016-01-01,16:00:00,0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,19:00:00,2016-01-01,19:00:00,0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,19:00:00,2016-01-01,19:00:00,0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,20:00:00,2016-01-01,16:00:00,0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,20:00:00,2016-01-01,01:00:00,0


In [80]:
data2 = {
    'airRes': data['airRes'],
    'hpgRes': data['hpgRes']
    }
# Group data by visit date  
for dfName in ['airRes', 'hpgRes']:
    data2[dfName] = data2[dfName].groupby(['air_store_id','visit_date'],as_index=False)[['reserve_visit_diff','reserve_visitors']].sum()


In [84]:
data2['hpgRes'].head(),"_"*30, data2['airRes'].head()

(           air_store_id  visit_date  reserve_visit_diff  reserve_visitors
 0  air_00a91d42b08b08d9  2016-01-14                   3                 2
 1  air_00a91d42b08b08d9  2016-01-15                   6                 4
 2  air_00a91d42b08b08d9  2016-01-16                   3                 2
 3  air_00a91d42b08b08d9  2016-01-22                   3                 2
 4  air_00a91d42b08b08d9  2016-01-29                   6                 5,
 '______________________________',
            air_store_id  visit_date  reserve_visit_diff  reserve_visitors
 0  air_00a91d42b08b08d9  2016-10-31                   0                 2
 1  air_00a91d42b08b08d9  2016-12-05                   4                 9
 2  air_00a91d42b08b08d9  2016-12-14                   6                18
 3  air_00a91d42b08b08d9  2016-12-17                   6                 2
 4  air_00a91d42b08b08d9  2016-12-20                   2                 4)

In [51]:
data['sampleSub'].head(10)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
5,air_00a91d42b08b08d9_2017-04-28,0
6,air_00a91d42b08b08d9_2017-04-29,0
7,air_00a91d42b08b08d9_2017-04-30,0
8,air_00a91d42b08b08d9_2017-05-01,0
9,air_00a91d42b08b08d9_2017-05-02,0


In [52]:
dfChk = data['hpgRes']
dfChk['visit_date'] = dfChk['visit_datetime'].dt.date
dfChk['visit_time'] = dfChk['visit_datetime'].dt.time
dfChk.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,visit_date,visit_time,reserve_date,reserve_time,reserve_visit_diff
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016-01-01,19:00:00,2016-01-01,15:00:00,0
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926,2016-01-02,19:00:00,2016-01-02,14:00:00,0
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926,2016-01-03,18:00:00,2016-01-02,20:00:00,1
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926,2016-01-06,20:00:00,2016-01-04,22:00:00,2
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926,2016-01-11,18:00:00,2016-01-11,14:00:00,0


In [56]:
dfRandomSub = data['sampleSub']
dfRandomSub.visitors = np.random.randint(1, 10, dfRandomSub.shape[0])
dfRandomSub.head(5)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,9
1,air_00a91d42b08b08d9_2017-04-24,6
2,air_00a91d42b08b08d9_2017-04-25,7
3,air_00a91d42b08b08d9_2017-04-26,9
4,air_00a91d42b08b08d9_2017-04-27,2


In [60]:
dfRandomSub.to_csv(PATH + r"\random_submission.csv", index = False)

In [58]:
len(dfRandomSub)

32019